# MCP

In [12]:
import os

import chromadb
import dotenv
from agents import Agent, Runner, function_tool, trace, WebSearchTool


dotenv.load_dotenv()

True

Let's set up our RAG database connection:

In [13]:
chroma_client = chromadb.PersistentClient(path="../chroma")
nutrition_db = chroma_client.get_collection(name="nutrition_db")

In [14]:
# This is the same code as in the rag.ipynb notebook


@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

Integrate WebSearchTool as Tool:

In [16]:


calorie_agent_with_search = Agent(
    name="Nutrition Assistant",
    instructions="""
    * You are a helpful nutrition assistant giving out calorie information.
    * You give concise answers.
    * You follow this workflow:
        0) First, use the calorie_lookup_tool to get the calorie information of the ingredients. But only use the result if it's explicitly for the food requested in the query.
        1) If you couldn't find the exact match for the food or you need to look up the ingredients, search the WebSearchTool to figure out the exact ingredients of the meal.
        Even if you have the calories in the web search response, you should still use the calorie_lookup_tool to get the calorie
        information of the ingredients to make sure the information you provide is consistent.
        2) Then, if necessary, use the calorie_lookup_tool to get the calorie information of the ingredients.
    * Even if you know the recipe of the meal, always use Exa Search to find the exact recipe and ingredients.
    * Once you know the ingredients, use the calorie_lookup_tool to get the calorie information of the individual ingredients.
    * If the query is about the meal, in your final output give a list of ingredients with their quantities and calories for a single serving. Also display the total calories.
    * Don't use the calorie_lookup_tool more than 10 times.
    """,
    tools=[calorie_lookup_tool,WebSearchTool()]
    
)

Reference query - shouldn't use webSearchTool:

In [17]:
with trace("Nutrition Assistant with WevSearchTool - Only uses calorie_lookup_tool"):
    result = await Runner.run(
        calorie_agent_with_search,
        "How many calories are in total in a banana and an apple? Also give calories per 100g",
    )
    print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    - Banana (medium, ~118 g): ~105 kcal
    - Apple (medium, ~182 g): ~95 kcal
    - Total for one banana + one apple: ~200 kcal
    - Calories per 100 g: Banana 89 kcal/100 g; Apple 52 kcal/100 g
- 7 new item(s)
- 3 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [18]:
with trace("Nutrition Assistant with WebSearchTool "):
    result = await Runner.run(
        calorie_agent_with_search, "How many calories are in an english breakfast?"
    )
    print(result.final_output)

A typical full English breakfast ranges roughly from 900 to 1,200 calories, depending on portion sizes. A common breakdown (per serving) is:

- 2 eggs (fried or grilled): ~180 kcal
- 2 rashers bacon: ~180–240 kcal
- 2 pork sausages: ~260–340 kcal
- Grilled tomato: ~20 kcal
- Sautéed/ fried mushrooms: ~40 kcal
- Baked beans (about 1/2 cup): ~120 kcal
- Toast (2 slices) with butter: ~180–240 kcal

Total: ~900–1,200 kcal

If you can tell me your exact portions (e.g., number of eggs, sausages, whether toast is buttered, and if you prefer fried vs. grilled), I’ll give a more precise total.
